In [ ]:
# Importing libraries
import os
import random
import gc
import warnings

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.data import Dataset
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, concatenate, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

warnings.filterwarnings('ignore')

# Preprocess Data

In [ ]:
# Importing the training data
Train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
Train_df.head()

In [ ]:
# Importing the test data
Test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
Test_id = Test_df.Id.copy()
Test_df.head()

In [ ]:
# Setting seeds
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

# Setting constants
batch_size = 32
image_size = 224
channels = 3
shuffle_size = 1024 

# Setting auto tune
AUTOTUNE = tf.data.experimental.AUTOTUNE  

In [ ]:
# Mapping the images ID to the image paths
Train_df.Id = Train_df.Id.map(lambda x: '../input/petfinder-pawpularity-score/train/' + x + '.jpg')
Test_df.Id = Test_df.Id.map(lambda x: '../input/petfinder-pawpularity-score/test/' + x + '.jpg')

In [ ]:
# Defining functions to decode image paths and preprocess images 
def read_img(labeled):
    def img_to_array(path):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=channels)
        image = tf.cast(image, tf.float32)
        image = tf.image.resize(image, (image_size, image_size))
        image = tf.keras.applications.efficientnet.preprocess_input(image)
        return image
    def mapping_train(path, struct_data, score):
        return (img_to_array(path),struct_data), score
    def mapping_test(path, struct_data):
        return (img_to_array(path),struct_data)
    return mapping_train if labeled else mapping_test

def augment(data, score):
    image, struct_data = data
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0.95, 1.05)
    image = tf.image.random_contrast(image, 0.95, 1.05)
    image = tf.image.random_brightness(image, 0.1)
    return (image, struct_data), score

def preprocess(ds, batch_size, ds_type, labeled):
    labeled_read_img = read_img(labeled)
    ds = ds.map(labeled_read_img, num_parallel_calls=AUTOTUNE)
    if ds_type=='train':
        ds = ds.map(augment, num_parallel_calls=AUTOTUNE)
        ds = ds.shuffle(shuffle_size, reshuffle_each_iteration=True)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(AUTOTUNE)
    return ds

In [ ]:
def create_ds(df, ds_type, labeled):
    ds = Dataset.from_tensor_slices((df['Id'].values,df.iloc[:,1:-1],df['Pawpularity'].values))
    ds = preprocess(ds, batch_size, ds_type, labeled)
    return ds

# Creating & Training models

The first model imported below i.e:```EffB0_1``` was obtained using the command ``` tf.keras.applications.efficientnet.EfficietNetB0(include_top=False)```, but here was imported from a Dataset so that we don't use internet in this kernel.

The second model imported below i.e:``` EffB0_2``` was obtained from a Github [repo](https://github.com/leondgarse/keras_efficientnet_v2#training-detail-from-article) under the name (EffV1B0), however it was imported here from a Dataset for the same reason cited above.

Both imported versions are of pretrained EfficientNetB0 on Imagenet and without the top of the neural network.


In [ ]:
# Importing EfficientNetB0 pretrained model obtained from Keras Applications API
EffNetB0_1_path = "../input/efficientnetb0-pretrained/EfficientNetB0.h5"
EffB0_1 = tf.keras.models.load_model(EffNetB0_1_path)
EffB0_1.trainable=False


# Importing EfficientNetB0 pretrained model obtained from a Github Repo
EffNetB0_2_path = "../input/efficientnetsv2-keras-notop-models/EfficientnetV1/Imagenet/efficientnetv1-b0-imagenet.h5"
EffB0_2 = tf.keras.models.load_model(EffNetB0_2_path)
EffB0_2.trainable=False

In [ ]:
# Defining the neural network model
# The named layers are going to be used to extract features for the catboost model training
def create_model(version):
    Inp1 = Input(shape=(image_size,image_size,channels))
    if version == '1':
        out1 = EffB0_1(Inp1)
    else:
        out1 = EffB0_2(Inp1)
        out1 = GlobalAveragePooling2D()(out1)
    out1 = Dropout(0.2)(out1)
    #out1 = BatchNormalization()(out1)
    out1 = Dense(16, activation='relu', kernel_initializer='he_normal')(out1)
    out1 = Dense(16, activation='relu', kernel_initializer='he_normal', name="Last_layer_Eff")(out1)

    Inp2 = Input(shape=(12,))
    out2 = Dense(16, activation='relu', kernel_initializer='he_normal', name="Last_layer_FFN")(Inp2)

    out = concatenate([out1,out2], axis=1)
    out = Dense(16, activation='relu', kernel_initializer='he_normal')(out)
    out = Dense(1, activation='relu')(out)

    PawModel = Model(inputs=[Inp1,Inp2], outputs=out)
    return PawModel

In [ ]:
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer._decayed_lr(tf.float32) # I use ._decayed_lr method instead of .lr
    return lr

**Training over 3 epochs only and using a third of the available data for a faster training and just to show the differences in the results of both models**

### Training using the first version

In [ ]:
#Training
Train = Train_df.iloc[:3000]
Val = Train_df.iloc[3000:3300]

Train_ds = create_ds(Train, ds_type='train', labeled=True)
Val_ds = create_ds(Val, ds_type='train', labeled=True)

model = create_model(version='1')

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
lr_metric = get_lr_metric(opt)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-6)
model.compile(loss='mse', 
          optimizer = opt, 
          metrics=[tf.keras.metrics.RootMeanSquaredError(), lr_metric])
results = model.fit(Train_ds,
                  epochs=3,
                  validation_data = Val_ds,
                  callbacks=[reduce_lr], verbose=1)
print('='*25)

# Freeing up memory
del model, results
del Train_ds, Val_ds
del Train, Val
gc.collect()

### Training using the second version under the same settings as the first version 

In [ ]:
#Training
Train = Train_df.iloc[:3000]
Val = Train_df.iloc[3000:3300]

Train_ds = create_ds(Train, ds_type='train', labeled=True)
Val_ds = create_ds(Val, ds_type='train', labeled=True)

model = create_model(version='2')

opt = tf.keras.optimizers.Adam(learning_rate=0.01)
lr_metric = get_lr_metric(opt)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, min_lr=1e-6)
model.compile(loss='mse', 
          optimizer = opt, 
          metrics=[tf.keras.metrics.RootMeanSquaredError(), lr_metric])
results = model.fit(Train_ds,
                  epochs=3,
                  validation_data = Val_ds,
                  callbacks=[reduce_lr], verbose=1)
print('='*25)

# Freeing up memory
del model, results
del Train_ds, Val_ds
del Train, Val
gc.collect()

**Both the validation, and training accuracy after 3 epochs of the second version is far worse from the ones of the first virsions!**